# Submitting Observations to the LCO Network

### Introduction
Most astronomers are used to taking observations by describing the exact instrument configuration and telescope pointing that they require, perhaps to a Telescope Operator, or via a Phase 2 system.  At LCO, our system allows you to describe the observation you need in the same way, and to submit that request directly.  

You can do this manually, using our [online portal](https://lco.global/observe), but its easy to write software to submit it for you and this interface offers a range of extra options which are not available through the web-form.  

This notebook is designed to show you how to use this Application Programmable Interface (API) to submit an observing request to LCO.  

### Example Science Case

ZTF18abchuyt<br>
RA = 00:45:31.4<br>
Dec = 59:59:52.9<br>
g = 15.992mag

Our target is a recent alert from ZTF.   The difference image shows a point-star image and the lightcurve shows a recent increase in all passbands.  The shape of its lightcurve over the next few weeks, measured in multiple passbands, will tell us about the physics taking place, so we request that the LCO 1m network observe this target once a day in ugriZ filters.  

### Describing the observation

This notebook uses a short Python script to submit this request for observations to the network, but you can write similar code in whatever language you prefer. 

We'll communicate with the network via HTTP, and Python provides a handy library to make this easy:

In [3]:
import requests

Now we'll describe the observation we want to make as a series of python dictionaries. 

Firstly, let's provide the details of the target, making sure to convert the coordinates to decimal degrees:

In [4]:
target = {
    'name': 'ZTF18abchuyt',
    'type': 'SIDEREAL',
    'ra': 11.3809479,
    'dec': 59.9980324,
    'epoch': 2000
}

Next we describe the exposures that we'd like to take.  

We want to use 1m telescope network to monitor the brightness of this object in ugriZ filters over the course of the next few weeks.  Firstly, we need to estimate what exposure time will be required in each filter, using LCO's online calculator which you can find [here](https://lco.global/files/etc/exposure_time_calculator.html).  

Exposures are always specified as an ordered *list of dictionaries*.  This enables you to take multi-passband sequences, just by requesting different filters.  For example, the code below would request 2 exposures in SDSS-u, followed by 2 in SDSS-g, followed by 2 in SDSS-r, and so on, just by adding dictionaries to the list.  Note the filter name abbreviations.  

In [26]:
exposures = [
    {
        'type': 'EXPOSE',
        'instrument_name': '1M0-SCICAM-SINISTRO',
        'filter': 'up',
        'exposure_time': 120,
        'exposure_count': 2,
        'bin_x': 1,
        'bin_y': 1,
        'defocus': 0.0,
        'ag_mode': 'OPTIONAL',
    },
    {
        'type': 'EXPOSE',
        'instrument_name': '1M0-SCICAM-SINISTRO',
        'filter': 'gp',
        'exposure_time': 60,
        'exposure_count': 2,
        'bin_x': 1,
        'bin_y': 1,
        'defocus': 0.0,
        'ag_mode': 'OPTIONAL',
    },
    {
        'type': 'EXPOSE',
        'instrument_name': '1M0-SCICAM-SINISTRO',
        'filter': 'rp',
        'exposure_time': 60,
        'exposure_count': 2,
        'bin_x': 1,
        'bin_y': 1,
        'defocus': 0.0,
        'ag_mode': 'OPTIONAL',
    },
    {
        'type': 'EXPOSE',
        'instrument_name': '1M0-SCICAM-SINISTRO',
        'filter': 'ip',
        'exposure_time': 60,
        'exposure_count': 2,
        'bin_x': 1,
        'bin_y': 1,
        'defocus': 0.0,
        'ag_mode': 'OPTIONAL',
    },
    {
        'type': 'EXPOSE',
        'instrument_name': '1M0-SCICAM-SINISTRO',
        'filter': 'zs',
        'exposure_time': 90,
        'exposure_count': 2,
        'bin_x': 1,
        'bin_y': 1,
        'defocus': 0.0,
        'ag_mode': 'OPTIONAL',
    }
]

The next step is to describe when we'd like this observation to start and finish, noting that all dates and times are specified in UTC.  The default would be for a single visit to the target, but in this case we also want the observation to be repeated once a day for several weeks.  

In [28]:
cadence = {
    'start': '2018-07-21 01:00:00',
    'end': '2018-09-30 23:59:59', 
    'period': 24.0, 
    'jitter': 12.0
}

During this period, the target may be visible from several sites in the network, at different times of day.  Working out the visibility windows from every telescope would be tedious!  So the network has an API which will do this for you.  

We just need to describe which telescopes we'd like to use.  In general, 
its best just to indicate which *aperture-class* of telescope you want - this tells the scheduler that it can assign this observation to *any* telescope in the network which is capable of performing the observation.  

The scheduler knows, in real-time, which telescopes are available, which are offline for engineering work, what the weather is at each telescope site, and where it will be dark during the time windows where you requested observations.  When you submit an observation, the scheduler works out the target visibility, and figures out which telescope sites can observe your target, within those windows.  

Of course, conditions at all the telescope sites can change.  The scheduler monitors this in real-time, and will re-schedule all observations, changing them from one telescope to another or even between sites if necessary.  For this reason, its always a good idea to give the scheduler the maximum possible flexibility in *where* and *when* it can carry out your observation.  

So we won't specify a location, except to say that in this case we'll use a 1m (rather than an 0.4m or a 2m) telescope. 

In [29]:
location = {
    'telescope_class': '1m0'
}

There are two other constraints which we may want to control - the maximum airmass that we will allow the telescope to point to and the minimum separation from the Moon.  The network provides sensible defaults, which we're doing to use here:

In [30]:
constraints = {
    'max_airmass': 1.6,
    'min_lunar_distance': 30
}


Of course, before we observe on any telescope, we first need to have an allocation of time!  Whenever you are successful in proposing to LCO, you'll be sent a proposal ID code, which represents the pot of time which will be automatically debited when observations are executed. So we'll need to include this in our observation request, to tell the scheduler which allocation to charge:

In [31]:
proposal_id = 'YOUR_PROPOSAL_ID'

Many projects have several targets which have observations ongoing simultaneously.  That's fine - the scheduler will try to do them all if it can.  But inevitably there are times when it can't, and in those cases its sometimes helpful for projects to indicate which of their own observations are most important to them.  This is called the observation's 'intra-proposal priority' and its specified as a floating point number.  'Normal' priority is 1.0, and you can read [here](https://lco.global/files/User_Documentation/the_new_priority_factor.pdf) about making use of this feature. 

In [32]:
ipp = 1.0

One last thing - its helpful to have a meaningful name for this observation so we can recognize it when we watch its progress through the [LCO observe portal](https://lco.global/observe).

In [33]:
group_id = 'ZTF1'

Now we can put all that together as a *user request*. 

In [45]:
cadence_request = {
    'group_id': group_id, 
    'proposal': proposal_id,
    'ipp_value': ipp,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'target': target,
        'molecules': exposures,
        'cadence': cadence,
        'location': location,
        'constraints': constraints
    }]
}

This request is not quite complete: first we need to ask the network when the target will be visible from the different sites.  

### Authentication

Since this is done over the wild west of the internet, to protect LCO from hostile attack, we need to establish some credentials.  

The good news is that you only need to do this once!  The network issues you a token which you can use from then onwards.  To get your token, go to [your profile page](https://observe.lco.global/accounts/profile/).  **But treat it like a password and don't share it!**  If you think its been compromise, you can revoke it from your profile page and get a new one.

A token looks like this:

In [46]:
token = "YOUR_TOKEN_HERE"

### Asking for windows for a cadence request

Adding these credentials to our user request, we can now send it to LCO as an HTTP request:

In [47]:
user_request = requests.post(
    'https://observe.lco.global/api/userrequests/cadence/',
    headers={'Authorization': 'Token '+token},
    json=cadence_request).json()

This API will return the request that you submitted, but you will find that the network has replaced the cadence dictionary we submitted with a list of specific windows when this target can be observed.

In [50]:
for r in user_request['requests']:
    print(r['windows'])


[{u'start': u'2018-08-02T19:00:00Z', u'end': u'2018-08-03T07:00:00Z'}]
[{u'start': u'2018-08-03T19:00:00Z', u'end': u'2018-08-04T07:00:00Z'}]
[{u'start': u'2018-08-04T19:00:00Z', u'end': u'2018-08-05T07:00:00Z'}]
[{u'start': u'2018-08-05T19:00:00Z', u'end': u'2018-08-06T07:00:00Z'}]
[{u'start': u'2018-08-06T19:00:00Z', u'end': u'2018-08-07T07:00:00Z'}]
[{u'start': u'2018-08-07T19:00:00Z', u'end': u'2018-08-08T07:00:00Z'}]
[{u'start': u'2018-08-08T19:00:00Z', u'end': u'2018-08-09T07:00:00Z'}]
[{u'start': u'2018-08-09T19:00:00Z', u'end': u'2018-08-10T07:00:00Z'}]
[{u'start': u'2018-08-10T19:00:00Z', u'end': u'2018-08-11T07:00:00Z'}]
[{u'start': u'2018-08-11T19:00:00Z', u'end': u'2018-08-12T07:00:00Z'}]
[{u'start': u'2018-08-12T19:00:00Z', u'end': u'2018-08-13T07:00:00Z'}]
[{u'start': u'2018-08-13T19:00:00Z', u'end': u'2018-08-14T07:00:00Z'}]
[{u'start': u'2018-08-14T19:00:00Z', u'end': u'2018-08-15T07:00:00Z'}]
[{u'start': u'2018-08-15T19:00:00Z', u'end': u'2018-08-16T07:00:00Z'}]
[{u'st

### Submitting the observation request

The request is now complete and ready to be submitted.  This is done using the same HTTP POST protocol, we just submit it to a different endpoint API, https://observe.lco.global/api/userrequests/ :

In [51]:
response = requests.post(
    'https://observe.lco.global/api/userrequests/',
    headers={'Authorization': 'Token '+token},
    json=user_request).json()

The network's reply will include your original request in JSON format, together with information on its status and any errors.  In case there was a problem, its worth parsing the reply and searching for the key word 'detail', which will tell you what the issue was.  If it was submitted successfully, the network will assign it a unique tracking number.  

In [54]:
if 'detail' in response.keys():
    print response['detail']
else:
    print('Tracking ID: '+str(response['id']))

Tracking ID: 675133


To monitor the progress of your observation, you can use the LCO observe portal and click on 'Requests' -> 'Submitted', or go straight to this particular request by replacing [track_id] in this URL: https://observe.lco.global/userrequests/[track_id]/

There are also programmatic ways to ask the network to tell you the status of your observation.  You can find out more about these at [developers.lco.global](https://developers.lco.global)